In [31]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from tqdm import tqdm
from transformers import AutoFeatureExtractor, ResNetForImageClassification
from sklearn.metrics import roc_curve
import numpy as np
import pandas as pd


In [32]:
# # Set the paths to your training, validation, and test directories
# train_dir = '/kaggle/input/small-dataset/train'
# val_dir = '/kaggle/input/small-dataset/val'
# test_dir = '/kaggle/input/small-dataset/test'

# # For the evaluation datasets
# fm_dir = '/kaggle/input/mad-benchmark-small/FaceMorpher'
# mg1_dir = '/kaggle/input/mad-benchmark-small/MIPGAN_I'
# mg2_dir = '/kaggle/input/mad-benchmark-small/MIPGAN_II'
# oc_dir = '/kaggle/input/mad-benchmark-small/OpenCV'
# wm_dir = '/kaggle/input/mad-benchmark-small/Webmorph'


In [33]:
# Set the paths to your training, validation, and test directories
train_dir = '/kaggle/input/morph-balanced/train'
val_dir = '/kaggle/input/morph-balanced/val'
# test_dir = '/kaggle/input/morph-splitted/test'

# For the evaluation datasets
fm_dir = '/kaggle/input/mad-benchmark/FaceMorpher'
mg1_dir = '/kaggle/input/mad-benchmark/MIPGAN_I'
mg2_dir = '/kaggle/input/mad-benchmark/MIPGAN_II'
oc_dir = '/kaggle/input/mad-benchmark/OpenCV'
wm_dir = '/kaggle/input/mad-benchmark/Webmorph'


In [34]:

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Function to reverse the class labels
def reverse_labels(label):
    return 1 - label

# Define transformations
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])


# Create datasets with reversed labels
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform, target_transform=reverse_labels)
val_dataset = datasets.ImageFolder(val_dir, transform=val_test_transform, target_transform=reverse_labels)
fm_dataset = datasets.ImageFolder(fm_dir, transform=val_test_transform, target_transform=reverse_labels)
mg1_dataset = datasets.ImageFolder(mg1_dir, transform=val_test_transform, target_transform=reverse_labels)
mg2_dataset = datasets.ImageFolder(mg2_dir, transform=val_test_transform, target_transform=reverse_labels)
oc_dataset = datasets.ImageFolder(oc_dir, transform=val_test_transform, target_transform=reverse_labels)
wm_dataset = datasets.ImageFolder(wm_dir, transform=val_test_transform, target_transform=reverse_labels)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
fm_loader = DataLoader(fm_dataset, batch_size=batch_size, shuffle=False)
mg1_loader = DataLoader(mg1_dataset, batch_size=batch_size, shuffle=False)
mg2_loader = DataLoader(mg2_dataset, batch_size=batch_size, shuffle=False)
oc_loader = DataLoader(oc_dataset, batch_size=batch_size, shuffle=False)
wm_loader = DataLoader(wm_dataset, batch_size=batch_size, shuffle=False)

In [35]:
# Define a self-attention layer
class SelfAttention(nn.Module):
    def __init__(self, in_dim):
        super(SelfAttention, self).__init__()
        self.query_conv = nn.Conv2d(in_dim, in_dim // 8, 1)
        self.key_conv = nn.Conv2d(in_dim, in_dim // 8, 1)
        self.value_conv = nn.Conv2d(in_dim, in_dim, 1)
        self.gamma = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        batch_size, C, width, height = x.size()
        proj_query = self.query_conv(x).view(batch_size, -1, width * height).permute(0, 2, 1)
        proj_key = self.key_conv(x).view(batch_size, -1, width * height)
        energy = torch.bmm(proj_query, proj_key)
        attention = nn.Softmax(dim=-1)(energy)
        proj_value = self.value_conv(x).view(batch_size, -1, width * height)

        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        out = out.view(batch_size, C, width, height)
        out = self.gamma * out + x
        return out

# Load pre-trained ResNet-50 model
base_model = models.resnet50(pretrained=True)

# Insert the attention layer before the final classifier
class ResNetWithAttention(nn.Module):
    def __init__(self, base_model):
        super(ResNetWithAttention, self).__init__()
        self.base_model = nn.Sequential(*list(base_model.children())[:-2])  # Exclude the final linear layer
        self.attention = SelfAttention(in_dim=2048)  # Attention layer with 2048 input channels
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(2048, 1)

    def forward(self, x):
        x = self.base_model(x)
        x = self.attention(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

model_with_attention = ResNetWithAttention(base_model)
model_with_attention.to(device)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model_with_attention.parameters(), lr=0.001)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [36]:
# Training loop
num_epochs = 10
best_model_wts = model_with_attention.state_dict()
best_acc = 0.0
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)

    for phase in ['train', 'val']:
        if phase == 'train':
            model_with_attention.train()  # Set model to training mode
        else:
            model_with_attention.eval()  # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        with tqdm(total=len(train_loader if phase == 'train' else val_loader), desc=f'{phase} Phase', unit='batch') as pbar:
            for inputs, labels in (train_loader if phase == 'train' else val_loader):
                inputs = inputs.to(device)
                labels = labels.float().view(-1, 1).to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model_with_attention(inputs)
                    preds = torch.sigmoid(outputs).round()
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                pbar.update(1)
                pbar.set_postfix(loss=running_loss / ((pbar.n + 1) * inputs.size(0)),
                                 accuracy=running_corrects.double() / ((pbar.n + 1) * inputs.size(0)))

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = model_with_attention.state_dict()

    print()

model_with_attention.load_state_dict(best_model_wts)
print('Best val Acc: {:4f}'.format(best_acc))


Epoch 1/10
----------


train Phase: 100%|██████████| 750/750 [07:48<00:00,  1.60batch/s, accuracy=tensor(0.9857, device='cuda:0', dtype=torch.float64), loss=0.0353]


train Loss: 0.0353 Acc: 0.9870


val Phase: 100%|██████████| 188/188 [01:31<00:00,  2.06batch/s, accuracy=tensor(1.9772, device='cuda:0', dtype=torch.float64), loss=0.0226]


val Loss: 0.0114 Acc: 0.9965

Epoch 2/10
----------


...

val Loss: 0.0064 Acc: 0.9978

Epoch 10/10
----------


train Phase: 100%|██████████| 750/750 [04:47<00:00,  2.60batch/s, accuracy=tensor(0.9919, device='cuda:0', dtype=torch.float64), loss=0.0221] 


train Loss: 0.0222 Acc: 0.9932


val Phase: 100%|██████████| 188/188 [00:47<00:00,  3.93batch/s, accuracy=tensor(1.9808, device='cuda:0', dtype=torch.float64), loss=0.00848]

val Loss: 0.0043 Acc: 0.9983

Best val Acc: 0.999167


In [37]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import roc_curve
from torchvision import models, transforms, datasets

# Ensure the classifier model is correctly defined and loaded
# Assuming 'classifier_model' is the model you want to evaluate
# model_with_attention should be already defined and trained

# Define evaluation function
def evaluate_model(model, data_loader, device):
    model.eval()
    true_labels = []
    predictions = []

    with torch.no_grad():
        for inputs, labels in tqdm(data_loader, desc='Evaluating', leave=False):
            inputs = inputs.to(device)
            outputs = model(inputs)
            probs = torch.sigmoid(outputs).cpu().numpy()
            true_labels.extend(labels.numpy())
            predictions.extend(probs)

    return np.array(true_labels), np.array(predictions)

# Define functions to calculate APCER, BPCER, EER, and accuracy
def calculate_apcer(true_labels, predictions, fixed_bpcer):
    fpr, tpr, _ = roc_curve(true_labels, predictions, pos_label=1)
    fpr_target = fixed_bpcer
    closest_fpr_index = np.argmin(np.abs(fpr - fpr_target))
    apcer = 1 - tpr[closest_fpr_index]
    return apcer

def calculate_bpcer(true_labels, predictions, fixed_apcer):
    fpr, tpr, _ = roc_curve(true_labels, predictions, pos_label=1)
    tpr_target = 1 - fixed_apcer
    closest_tpr_index = np.argmin(np.abs(tpr - tpr_target))
    bpcer = fpr[closest_tpr_index]
    return bpcer

def calculate_eer(true_labels, predictions):
    fpr, tpr, _ = roc_curve(true_labels, predictions, pos_label=1)
    eer_index = np.argmin(np.abs(fpr - (1 - tpr)))
    eer = fpr[eer_index]
    return eer

def calculate_accuracy(true_labels, predictions):
    binary_predictions = (predictions > 0.5).astype(int)
    accuracy = np.mean(true_labels == binary_predictions)
    return accuracy

# Compute metrics for a dataset
def compute_metrics_for_dataset(model, data_loader, device, fixed_bpcer_values, fixed_apcer_values):
    true_labels, predictions = evaluate_model(model, data_loader, device)
    metrics = {
        'APCER': {bpcer: calculate_apcer(true_labels, predictions, bpcer) for bpcer in fixed_bpcer_values},
        'BPCER': {apcer: calculate_bpcer(true_labels, predictions, apcer) for apcer in fixed_apcer_values},
        'EER': calculate_eer(true_labels, predictions),
        'Accuracy': calculate_accuracy(true_labels, predictions)
    }
    return metrics

# Define fixed values for APCER and BPCER calculations
fixed_bpcer_values = [0.01, 0.1, 0.2]
fixed_apcer_values = [0.01, 0.1, 0.2]

# Create a DataFrame to store results
results_df = pd.DataFrame(columns=['Dataset', 'APCER_0.01', 'APCER_0.1', 'APCER_0.2',
                                   'BPCER_0.01', 'BPCER_0.1', 'BPCER_0.2', 'EER', 'Accuracy'])

# Loaders dictionary
loaders = {
    'FaceMorpher': fm_loader,
    'MIPGAN_I': mg1_loader,
    'MIPGAN_II': mg2_loader,
    'OpenCV': oc_loader,
    'Webmorph': wm_loader
}

# Iterate through the loaders and compute metrics for each dataset
for dataset_name, loader in loaders.items():
    metrics = compute_metrics_for_dataset(model_with_attention, loader, device, fixed_bpcer_values, fixed_apcer_values)
    
    # Extract APCER and BPCER values for each fixed threshold
    apcer_0_01 = metrics['APCER'][0.01]
    apcer_0_1 = metrics['APCER'][0.1]
    apcer_0_2 = metrics['APCER'][0.2]
    
    bpcer_0_01 = metrics['BPCER'][0.01]
    bpcer_0_1 = metrics['BPCER'][0.1]
    bpcer_0_2 = metrics['BPCER'][0.2]
    
    # Create a DataFrame for the current dataset metrics
    df = pd.DataFrame({'Dataset': [dataset_name],
                       'APCER_0.01': [apcer_0_01],
                       'APCER_0.1': [apcer_0_1],
                       'APCER_0.2': [apcer_0_2],
                       'BPCER_0.01': [bpcer_0_01],
                       'BPCER_0.1': [bpcer_0_1],
                       'BPCER_0.2': [bpcer_0_2],
                       'EER': [metrics['EER']],
                       'Accuracy': [metrics['Accuracy']]})

    # Concatenate the current dataset DataFrame with the results_df
    results_df = pd.concat([results_df, df], ignore_index=True)

results_df.head()

/tmp/ipykernel_34/1828690681.py:109: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, df], ignore_index=True)


,Dataset,APCER_0.01,APCER_0.1,APCER_0.2,BPCER_0.01,BPCER_0.1,BPCER_0.2,EER,Accuracy
0,FaceMorpher,0.784314,0.450980,0.235294,0.582000,0.325000,0.232000,0.217000,0.815739
1,MIPGAN_I,0.759804,0.514706,0.392157,0.956000,0.675000,0.501000,0.313000,0.815739
2,MIPGAN_II,0.661765,0.397059,0.279412,0.842843,0.457457,0.300300,0.243243,0.815592
3,OpenCV,0.401961,0.142157,0.083333,0.552846,0.135163,0.059959,0.119919,0.813361
4,Webmorph,0.460784,0.151961,0.088235,0.568000,0.152000,0.080000,0.134000,0.694102


In [38]:
import torch
import numpy as np
from sklearn.metrics import confusion_matrix
from tqdm import tqdm

# Assuming `model_with_attention` is your trained model
# and `device` is already defined (e.g., `device = torch.device("cuda" if torch.cuda.is_available() else "cpu")`)

# Function to calculate the confusion matrix
def calculate_confusion_matrix(model, data_loader, device):
    true_labels = []
    predicted_labels = []

    model.eval()
    with torch.no_grad():
        for inputs, labels in tqdm(data_loader, desc='Calculating Confusion Matrix', leave=False):
            inputs = inputs.to(device)
            labels = labels.float().view(-1, 1).to(device)

            outputs = model(inputs)
            preds = (torch.sigmoid(outputs) > 0.5).float()

            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(preds.cpu().numpy())

    true_labels = np.concatenate(true_labels)
    predicted_labels = np.concatenate(predicted_labels)

    confusion_mat = confusion_matrix(true_labels, predicted_labels)
    return confusion_mat

# Assuming `loaders` is a dictionary of data loaders defined earlier
data_loaders = [fm_loader, mg1_loader, mg2_loader, oc_loader, wm_loader]
dataset_names = ["FaceMorpher", "MIPGAN_I", "MIPGAN_II", "OpenCV", "Webmorph"]

confusion_matrices = []

# Calculate and store confusion matrices for each dataset
for loader_idx, data_loader in enumerate(data_loaders):
    dataset_name = dataset_names[loader_idx]
    print(f"Calculating confusion matrix for dataset: {dataset_name}")

    confusion_mat = calculate_confusion_matrix(model_with_attention, data_loader, device)
    print(confusion_mat)
    confusion_matrices.append(confusion_mat)

# Print the confusion matrices
# for dataset_name, confusion_mat in zip(dataset_names, confusion_matrices):
#     print(f"\nConfusion Matrix - {dataset_name}:")
#     print(confusion_mat)


Calculating confusion matrix for dataset: FaceMorpher


[[1000    0]
 [ 177   27]]
Calculating confusion matrix for dataset: MIPGAN_I


[[1000    0]
 [ 177   27]]
Calculating confusion matrix for dataset: MIPGAN_II


[[999   0]
 [177  27]]
Calculating confusion matrix for dataset: OpenCV


[[984   0]
 [177  27]]
Calculating confusion matrix for dataset: Webmorph


[[500   0]
 [177  27]]


In [39]:
# import torch
# from torchvision import datasets
# from torch.utils.data import DataLoader

# # Function to count images per class in a dataset
# def count_images_per_class(dataset):
#     class_counts = {}
#     for _, label in dataset.samples:
#         class_name = dataset.classes[label]
#         if class_name in class_counts:
#             class_counts[class_name] += 1
#         else:
#             class_counts[class_name] = 1
#     return class_counts

# # List of all datasets
# datasets_list = [
#     ('Train', train_dataset),
#     ('Validation', val_dataset),
#     ('FM', fm_dataset),
#     ('MG1', mg1_dataset),
#     ('MG2', mg2_dataset),
#     ('OC', oc_dataset),
#     ('WM', wm_dataset),
# ]

# # Print number of images per class for each dataset
# for name, dataset in datasets_list:
#     class_counts = count_images_per_class(dataset)
#     print(f'{name} Dataset:')
#     for class_name, count in class_counts.items():
#         print(f'  Class: {class_name}, Number of images: {count}')
#     print()
